In [ ]:
def load_image(image_path):
    """Load and convert an image to grayscale.
    Returns:
        numpy.ndarray: Grayscale image array with values 0-255
    """
    try:
        with Image.open(image_path) as img:
            if img.mode != 'L':
                img = img.convert('L')
            # Convert to numpy array and ensure correct dtype
            return np.array(img, dtype=np.uint8)
    except Exception as e:
        raise ValueError(f"Error processing image: {str(e)}")

def display_images(images, titles=None, figsize=(15, 5), cmap='gray',border_color='black', border_width=1):
    """Display images in a truly compact grid layout with maximum 4 images per row.
    
    This function creates a highly compact display by carefully controlling both the
    figure size and the spacing between subplots. It uses a combination of GridSpec
    and figure size calculations to ensure images are displayed with minimal gaps
    while maintaining readability.
    
    Args:
        images (list): List of images to display
        titles (list): Optional list of titles for each image
        figsize (tuple): Base figure size (width, height) used as a reference for calculations
        cmap (str): Colormap to use for displaying images
    """
    n = len(images)
    if titles is None:
        titles = [f'Image {i+1}' for i in range(n)]
    
    # Calculate grid dimensions
    cols = min(n, 4)
    rows = (n + cols - 1) // cols
    
    # Calculate figure size differently to ensure compact layout
    # Use the aspect ratio of the first image to maintain proportions
    sample_image = images[0]
    aspect_ratio = sample_image.shape[0] / sample_image.shape[1]
    
    # Base width calculation considering the number of columns
    base_unit = 4  # Base unit for size calculations
    fig_width = base_unit * cols
    fig_height = base_unit * aspect_ratio * rows
    
    # Create figure and GridSpec with tight spacing
    fig = plt.figure(figsize=(fig_width, fig_height))
    gs = plt.GridSpec(rows, cols, figure=fig,
                     left=0.01, right=0.99,
                     bottom=0.01, top=0.90,
                     wspace=0.01, hspace=0.15)
    
    # Create and populate subplots
    for idx in range(n):
        row = idx // cols
        col = idx % cols
        
        # Create subplot
        ax = fig.add_subplot(gs[row, col])
        
        # Display image without interpolation for sharper display
        ax.imshow(images[idx], cmap=cmap, interpolation='nearest')
        ax.set_title(titles[idx], pad=2, fontsize=14)
        
        # Remove all axes elements
        ax.set_xticks([])
        ax.set_yticks([])
        for spine in ax.spines.values():
                    spine.set_visible(True)  # Make sure spine is visible
                    spine.set_color(border_color)  # Set border color
                    spine.set_linewidth(border_width)  # Set border width
    
    plt.show()

def analyze_image(image):
    """Analyze basic characteristics of an image.
    
    Computes and returns useful statistics about the image:
    - Min and max pixel values
    - Mean and median intensity
    - Standard deviation of pixel values
    - Histogram data
    
    Returns:
        dict: Dictionary containing image statistics
    """
    stats = {
        'min': int(np.min(image)),
        'max': int(np.max(image)),
        'mean': float(np.mean(image)),
        'median': float(np.median(image)),
        'std': float(np.std(image))
    }
    
    # Compute histogram
    hist, bins = np.histogram(image.flatten(), bins=256, range=[0, 256])
    stats['histogram'] = hist
    stats['bins'] = bins
    
    return stats

def plot_histogram(image, title="Image Histogram", figsize=(10, 4)):
    """Plot the histogram of a grayscale image.
    
    Creates a visualization of the image's intensity distribution,
    which is crucial for understanding how to set thresholds.
    
    Args:
        image (numpy.ndarray): Input grayscale image
        title (str): Title for the plot
        figsize (tuple): Figure size
    """
    plt.figure(figsize=figsize)
    plt.hist(image.ravel(), bins=256, range=[0, 256], density=True, alpha=0.7)
    plt.title(title)
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.show()

def process_random_image(folder_path, algorithms, num_images=1, seed=None):
    """Process random images from a directory using specified algorithms.
    
    This function streamlines our testing workflow by:
    1. Randomly selecting images from a directory
    2. Applying multiple processing algorithms
    3. Displaying results side by side for comparison
    
    Args:
        folder_path (str): Path to the directory containing images
        algorithms (dict): Dictionary of {name: function} pairs for processing
                         Each function should take an image array as input
        num_images (int): Number of random images to process
        seed (int): Random seed for reproducibility
    
    Returns:
        list: List of processed image sets for further analysis if needed
    
    Example usage:
        algorithms = {
            'Basic Threshold': lambda img: basic_threshold(img, 127),
            'Adaptive': lambda img: adaptive_threshold(img)
        }
        process_random_image('../images', algorithms)
    """
    # Set random seed if provided
    if seed is not None:
        random.seed(seed)
    
    # Get list of valid image files
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    image_files = [
        f for f in os.listdir(folder_path) 
        if f.lower().endswith(valid_extensions)
    ]
    
    if not image_files:
        raise ValueError(f"No valid images found in {folder_path}")
    
    # Process specified number of random images
    results = []
    for _ in range(num_images):
        image_file = random.choice(image_files)
        image_path = os.path.join(folder_path, image_file)
        
        original = load_image(image_path)
        
        # Apply each algorithm
        processed_images = [original]
        titles = ['Original']
        
        for name, algo in algorithms.items():
            try:
                processed = algo(original)
                processed_images.append(processed)
                titles.append(name)
            except Exception as e:
                print(f"Error applying {name}: {str(e)}")
        
        print(f"\nProcessing: {image_file}")
        
        plot_histogram(original, f"Histogram for {image_file}")
        
        display_images(
            processed_images,
            titles,
            figsize=(4 * len(processed_images), 4)
        )
        
        stats = analyze_image(original)
        print("\nImage Statistics:")
        print(f"Dimensions: {original.shape}")
        print(f"Intensity Range: {stats['min']} - {stats['max']}")
        print(f"Mean Intensity: {stats['mean']:.2f}")
        print(f"Standard Deviation: {stats['std']:.2f}")
        
        results.append({
            'filename': image_file,
            'original': original,
            'processed': processed_images[1:],
            'stats': stats
        })
    
    return results   

## Pixel Processing
- Functions for basic transformation
- Brihgtness, Contrast, and Gamma
- Per-pixel Operation